todo
1. optimize hyperparameters
2. clean dataset
    - balance
3. data augmentation
4. add requirments.txt
5. correct title (of file as well)
6. go through all variable names
7. ask which types of groups should "be in focus". Do disabled people count?
8. more advaned duplicate dropping?
9. balancing the dataset needed?
10. dedicated train test split?
11. data augmentation only on train?
12. remove duplicates a second time

# Hate Detection

In [214]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix, f1_score

from keras.callbacks import EarlyStopping
from keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


from random import sample

### Data Exploration

In [215]:
data = pd.read_csv('hate-text.csv')
data

,case_id,test_case,label_gold,target_ident
0,1,I hate women.,hateful,women
1,2,I hate trans people.,hateful,trans people
2,3,I hate gay people.,hateful,gay people
3,4,I hate black people.,hateful,black people
4,5,I hate disabled people.,hateful,disabled people
...,...,...,...,...
3896,3897,Gay people are all complete 4ssholes.,hateful,gay people
3897,3898,Black people are all complete 4ssholes.,hateful,black people
3898,3899,Disabled people are all complete 4ssholes.,hateful,disabled people
3899,3900,Muslims are all complete 4ssholes.,hateful,Muslims


The dataset contains some missing values in the target_ident column, which we'll have to deal with later

In [216]:
# Using display() to display both at the same time. Won't comment any future use of display() (for obvious reasons)
display(
    data.isnull().any(),
    data[data.isna().any(axis=1)]
)

case_id         False
test_case       False
label_gold      False
target_ident     True
dtype: bool

,case_id,test_case,label_gold,target_ident
1271,1272,"Fucking hell, what a day.",non-hateful,NaN
1272,1273,That's a great fucking idea.,non-hateful,NaN
1273,1274,No fucking way.,non-hateful,NaN
1274,1275,I'm so fucking excited.,non-hateful,NaN
1275,1276,Are you fucking kidding me?,non-hateful,NaN
...,...,...,...,...
3097,3098,Pro lifers should get the fuck out of our coun...,non-hateful,NaN
3098,3099,Most gun dealers are seriously fucked in the h...,non-hateful,NaN
3099,3100,"By nature, corporate accountants are just shit...",non-hateful,NaN
3100,3101,Most real estate agents are absolute bastards.,non-hateful,NaN


No duplicate texts found

In [217]:
print(f'''duplicate texts: {data['test_case'].duplicated().any()}''')

duplicate texts: False


While the dataset is quite balanced in the target groups, it's still assymetric when it comes to the number of hate/non-hate texts

In [218]:
display(
    pd.DataFrame(data['label_gold'].value_counts()),
    pd.DataFrame(data['target_ident'].value_counts())
)

,count
label_gold,
hateful,2659
non-hateful,1242


,count
target_ident,
gay people,577
women,535
disabled people,510
Muslims,510
black people,504
trans people,485
immigrants,485


No unusually short/long texts

In [219]:
print(
    f'Shortest: {len(data["test_case"].iloc[0])}\n'
    f'Longest: {len(data["test_case"].iloc[-1])}'
)

Shortest: 14
Longest: 38


### Data preprocessing

Drop and rename columns to something more suitable

In [220]:
data = data.drop('case_id', axis=1)
data = data.rename(
    columns=
    {
        'test_case': 'text',
        'label_gold': 'label',
        'target_ident': 'target',
    }
)

data.head()

,text,label,target
0,I hate women.,hateful,women
1,I hate trans people.,hateful,trans people
2,I hate gay people.,hateful,gay people
3,I hate black people.,hateful,black people
4,I hate disabled people.,hateful,disabled people


It appears as though non of the missing target data have any relevant hate speech associated with them. This can easily be varified by manually looking at the dataset (given its tiny size). But alas, given my horrendous dyslexia i decided to programmatically varify it as a sanity check

In [221]:
target_search_words = [
    'gay',
    'women',
    'disabled',
    'Muslims',
    'black',
    'trans',
    'immigrants'
]

# Get all rows targets with NaNs
data_target_nans = data[data['target'].isna()]

for index, row in data_target_nans.iterrows():
    for word in target_search_words:
        # Split by words and search each one
        if word in row['text'].split(' '):
            print(f'''found target word in: "{row['text']}" row''')

I'm now confident in dropping all rows with target NaN values

In [222]:
data = data.dropna()

Balance the dataset

In [223]:
hateful = data['label'].value_counts()['hateful']
non_hateful = data['label'].value_counts()['non-hateful']

# Because we've got far more hateful texts
number_of_rows_to_drop = hateful - non_hateful

# randomly sample the rows and and drop them
rows_indexes_to_remove = data[data['label'] == 'hateful'].sample(n=number_of_rows_to_drop).index
data = data.drop(rows_indexes_to_remove)

display(
    pd.DataFrame(data['label'].value_counts()),
    pd.DataFrame(data['target'].value_counts())
)

,count
label,
hateful,947
non-hateful,947


,count
target,
gay people,333
women,296
disabled people,282
black people,255
immigrants,251
trans people,240
Muslims,237


### Data Augmentation

In [224]:
def shuffle_sentences(row):
    word_list = row['text'].split(' ')
    shuffled_sentance = ' '.join(sample(word_list, len(word_list)))
    return shuffled_sentance

shuffled_sentences = data.apply(shuffle_sentences, axis=1)

data_copy = data.copy()
data_copy = data_copy.drop('text', axis=1)
data_copy.insert(1, 'text', shuffled_sentences)

data = pd.concat([data, data_copy], ignore_index=True)

data

,text,label,target
0,I hate women.,hateful,women
1,I hate gay people.,hateful,gay people
2,I hate black people.,hateful,black people
3,I hate immigrants.,hateful,immigrants
4,I really can't stand women like you.,hateful,women
...,...,...,...
7571,worth Immigrant aren't sh1t. lives,hateful,immigrants
7572,4ssholes. all are complete Women,hateful,women
7573,4ssholes. Gay people are complete all,hateful,gay people
7574,all are people Black 4ssholes. complete,hateful,black people


Shuffle the dataset

In [225]:
# Shuffles and returns the entire dataset as frac=1
data = data.sample(frac=1)

# Reset index after having both balanced and shuffled
data = data.reset_index(drop=True)
data

,text,label,target
0,trans all. once Let's and people for rid get of,hateful,trans people
1,think those to to kinds ought wrong Am of immi...,hateful,immigrants
2,I cannot believe this queer.,hateful,gay people
3,a you are about small-minded Writing says tha...,non-hateful,Muslims
4,people fantastic. are Trans,non-hateful,trans people
...,...,...,...
7571,are All scumbags. such massive women,hateful,women
7572,fags bad so If only smoking lungs. your for w...,non-hateful,gay people
7573,I'll cherish any trans person like you.,non-hateful,trans people
7574,If you then hate say bigot! you are black peop...,non-hateful,black people


In [226]:
max_words_to_use = 1000

# Tokenize the text data (convert them into "sequences")
tokenizer = Tokenizer(num_words=max_words_to_use) # Consider only using the top 1000 words, as those 
tokenizer.fit_on_texts(data['text'])
tokenized_data = tokenizer.texts_to_sequences(data['text'])

# printing using loop for easier viewing
for i in range(5):
    print(f'seq {i}: {tokenized_data[i]}')

seq 0: [15, 8, 211, 139, 39, 1, 20, 199, 50, 18]
seq 1: [100, 33, 3, 3, 140, 93, 547, 58, 18, 23, 117, 6, 11, 9]
seq 2: [6, 200, 85, 30, 114]
seq 3: [5, 4, 2, 106, 297, 298, 201, 299, 11, 56, 8, 118, 26, 162, 2]
seq 4: [1, 300, 2, 15]
